In [15]:
import re
from datetime import datetime

def parse_log_a(file_path):
    tag_energy_per_month = {}
    current_tag = None
    session_start_meter = None
    current_power = 0.0
    last_time = None

    with open(file_path, 'r') as file:
        for line in file:
            # Extract tag ID
            tag_match = re.search(r'tag: (\w+)', line)
            if tag_match:
                new_tag = tag_match.group(1)
                if new_tag != current_tag:
                    current_tag = new_tag
                continue

            # Extract start of new charging session
            session_start = re.search(r'New charging session started, metervalue=(\d+\.\d+)', line)
            if session_start:
                if session_start_meter is not None:
                    # Calculate kWh for the previous session if it's not the first session
                    energy = (float(session_start.group(1)) - session_start_meter) / 1000.0  # Convert to kWh
                    month = last_time.strftime('%Y-%m')
                    if current_tag not in tag_energy_per_month:
                        tag_energy_per_month[current_tag] = {}
                    if month not in tag_energy_per_month[current_tag]:
                        tag_energy_per_month[current_tag][month] = 0
                    tag_energy_per_month[current_tag][month] += energy

                # Update the meter value for the new session
                session_start_meter = float(session_start.group(1))
                continue

            # Extract time for tracking
            time_match = re.search(r'(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2})', line)
            if time_match:
                last_time = datetime.strptime(time_match.group(1), '%Y-%m-%dT%H:%M:%S')

    # Handle the last session if there was one
    if session_start_meter is not None and last_time:
        # Assuming the meter didn't change after the last session start
        energy = 0  # Since we don't have an end meter value, we'll set it to 0 or use the last known meter value if available
        month = last_time.strftime('%Y-%m')
        if current_tag not in tag_energy_per_month:
            tag_energy_per_month[current_tag] = {}
        if month not in tag_energy_per_month[current_tag]:
            tag_energy_per_month[current_tag][month] = 0
        tag_energy_per_month[current_tag][month] += energy  # This will be 0 unless we have another way to estimate the energy

    return tag_energy_per_month



In [18]:
# Path to your log file
file_path = './data/laadsessies/NLSHOE00025967-ACE0149315-04.02.2025_15.18-04.02.2025_15.18.txt'

# Parse the log
energy_per_tag_per_month = parse_log_a(file_path)

# Print results
for tag, months in energy_per_tag_per_month.items():
    print(f"Tag {tag}:")
    for month, energy in months.items():
        print(f"  {month}: {energy:.3f} kWh")
    print()  # Add a line break for readability

Tag 04AE216A2C5D:
  2024-12: 10.064 kWh
  2025-01: 94.739 kWh
  2025-02: 10.704 kWh

Tag 04AE216A2C5:
  2025-02: 0.000 kWh



In [17]:
# Path to your log file
file_path = './data/laadsessies/NLSHOE00025967-ACE0149315-17.03.2025_19.44-17.03.2025_19.44.txt'

# Parse the log
energy_per_tag_per_month = parse_log_a(file_path)

# Print results
for tag, months in energy_per_tag_per_month.items():
    print(f"Tag {tag}:")
    for month, energy in months.items():
        print(f"  {month}: {energy:.3f} kWh")
    print()  # Add a line break for readability

Tag 04AE216A2C5D:
  2025-02: 93.531 kWh
  2025-03: 59.518 kWh

Tag 04AE216A2C5D80:
  2025-02: 6.592 kWh

Tag 04AE216A2C5:
  2025-03: 0.000 kWh



In [19]:
import re
from datetime import datetime

def parse_log(file_path):
    # Dictionary to hold metervalues and timestamps per tag
    tag_sessions = {}
    # Dictionary to hold final energy per tag per month
    tag_energy_per_month = {}

    current_tag = None

    with open(file_path, 'r') as file:
        for line in file:
            # Extract tag ID
            tag_match = re.search(r'tag: (\w+)', line)
            if tag_match:
                current_tag = tag_match.group(1)
                if current_tag not in tag_sessions:
                    tag_sessions[current_tag] = []
                continue

            # Extract metervalue from session start
            metervalue_match = re.search(r'New charging session started, metervalue=(\d+\.\d+)', line)
            if metervalue_match and current_tag:
                time_match = re.search(r'(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2})', line)
                if time_match:
                    session_time = datetime.strptime(time_match.group(1), '%Y-%m-%dT%H:%M:%S')
                    metervalue = float(metervalue_match.group(1))
                    tag_sessions[current_tag].append((session_time, metervalue))

    # Calculate energy usage between sessions for each tag
    for tag, sessions in tag_sessions.items():
        if len(sessions) < 2:  # Need at least 2 sessions to calculate differences
            continue

        tag_energy_per_month[tag] = {}
        for i in range(len(sessions) - 1):
            start_time, start_metervalue = sessions[i]
            end_time, end_metervalue = sessions[i + 1]

            # Calculate energy in kWh (metervalues are in Wh, so divide by 1000 for kWh)
            energy_kwh = (end_metervalue - start_metervalue) / 1000.0

            # Attribute energy to the month of the start time
            month = start_time.strftime('%Y-%m')
            if month not in tag_energy_per_month[tag]:
                tag_energy_per_month[tag][month] = 0
            tag_energy_per_month[tag][month] += energy_kwh

    return tag_energy_per_month



In [12]:
# Path to your log file
file_path = './data/laadsessies/NLSHOE00025967-ACE0149315-04.02.2025_15.18-04.02.2025_15.18.txt'

# Parse the log
energy_per_tag_per_month = parse_log(file_path)

# Print results
for tag, months in energy_per_tag_per_month.items():
    print(f"Tag {tag}:")
    for month, energy in months.items():
        print(f"  {month}: {energy:.3f} kWh")
    print()  # Add a line break for readability

Tag 04AE216A2C5D:
  2024-12: 14.738 kWh
  2025-01: 96.599 kWh
  2025-02: 4.170 kWh



In [22]:
# Path to your log file
file_path = './data/laadsessies/NLSHOE00025967-ACE0149315-17.03.2025_19.44-17.03.2025_19.44.txt'

# Parse the log
energy_per_tag_per_month = parse_log(file_path)

# Print results
for tag, months in energy_per_tag_per_month.items():
    print(f"Tag {tag}:")
    for month, energy in months.items():
        print(f"  {month}: {energy:.3f} kWh")
    print()  # Add a line break for readability

Tag 04AE216A2C5D:
  2025-02: 105.130 kWh
  2025-03: 54.511 kWh



In [14]:
import pandas as pd
import glob

path = './data/laadsessies/'

all_files = glob.glob(path + "Coolblue laadsessies *.csv")

dfs = []

for filename in all_files:
    df = pd.read_csv(filename)
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df.to_csv(path + 'all_load_sessions.csv', index=False)
combined_df['Starttijd'] = pd.to_datetime(combined_df['Starttijd'])
combined_df['Month'] = combined_df['Starttijd'].dt.to_period('M')
monthly_kWh = combined_df.groupby('Month')['Opgeladen kWh'].sum()
monthly_kWh_formatted = monthly_kWh.apply(lambda x: f"{x:,.3f}".replace('.', ','))
monthly_kWh_df = pd.DataFrame({
    'Opgeladen kWh': monthly_kWh,
    'Opgeladen kWh (comma)': monthly_kWh_formatted
})

monthly_kWh_df



,Opgeladen kWh,Opgeladen kWh (comma)
Month,,
2023-11,105.767,"105,767"
2023-12,128.217,"128,217"
2024-01,186.616,"186,616"
2024-02,161.444,"161,444"
2024-03,195.758,"195,758"
2024-04,174.622,"174,622"
2024-05,182.136,"182,136"
2024-06,121.849,"121,849"
2024-07,145.597,"145,597"
